In [0]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

# Light.

### Задача.

Создайте модель для распознавания рукописных цифр из набора MNIST (можно воспользоваться ноутбуком 1-го занятия) и проведите ряд тестов: 
1. Запустите сеть с различными размерами обучающей и проверочной выборок: 
- Обучающая выборка 50.000 примеров 
- Обучающая выборка 10.000 примеров 
- Обучающая выборка 500 примеров 
2. Создайте еще два варианта сети и сравните значения точности на проверочной выборке (на 
последней эпохе) и на тестовой выборке. Сделайте сравнительную таблицу. 
3. Создайте сеть следующей архитектуры: 
- 4 Dense слоя 
- 3 Dropout слоя
- 3 BatchNormalization слоя 

Напишите свои выводы по результатам проведенных тестов. 


### Решение.

In [0]:
# prepare data
(x_train, y_train_org), (x_test, y_test_org) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_train = x_train / 255
x_test = x_test.astype('float32')
x_test = x_test / 255
y_train = utils.to_categorical(y_train_org, 10)
y_test = utils.to_categorical(y_test_org, 10)

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# task 1
list_sample = [50000, 10000, 500]
string = 0
table_vars = pd.DataFrame(columns = ['size_of_sample', 'dropout_layers', 'batch_norm_layers', 'accuracy_train', 'accuracy_test'])
for sample in list_sample:
  model = Sequential()
  model.add(Dense(800, input_dim = 784, activation='relu'))
  model.add(Dense(100, activation='linear'))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(x_train[:sample+1], y_train[:sample+1], batch_size=768, epochs=30, verbose=1)
  res_base = model.evaluate(x_test, y_test, verbose=1)
  table_vars.loc[string] = [sample, 0, 0, history.history['accuracy'][-1], res_base[1]]
  string += 1

In [0]:
# task 2
# network with dropout layers
model = Sequential()
model.add(Dropout(0.3, input_shape=(784,)))
model.add(Dense(800, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='linear'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train[:50001], y_train[:50001], batch_size=768, epochs=30, verbose=1)
# I left 50,000 for equal conditions 
res_base = model.evaluate(x_test, y_test, verbose=1)
table_vars.loc[string] = [50000, 3, 0, history.history['accuracy'][-1], res_base[1]]
string += 1

In [0]:
# network with batchnormalization layers
model = Sequential()
model.add(BatchNormalization(input_shape=(784, )))
model.add(Dense(800, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(100, activation='linear'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(100, activation='linear'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train[:50001], y_train[:50001], batch_size=768, epochs=30, verbose=1)
res_base = model.evaluate(x_test, y_test, verbose=1)
table_vars.loc[string] = [50000, 3, 3, history.history['accuracy'][-1], res_base[1]]
string += 1

In [0]:
# task 3
model = Sequential()
model.add(BatchNormalization(input_shape=(784, )))
model.add(Dense(800, input_dim = 784, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='linear'))
model.add(BatchNormalization())
model.add(Dense(100, activation='linear'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train[:50001], y_train[:50001], batch_size=768, epochs=30, verbose=1)
res_base = model.evaluate(x_test, y_test, verbose=1)
table_vars.loc[string] = [50000, 0, 3, history.history['accuracy'][-1], res_base[1]]
string += 1

In [0]:
table_vars

,size_of_sample,dropout_layers,batch_norm_layers,accuracy_train,accuracy_test
0,50000.0,0.0,0.0,0.99924,0.9768
1,10000.0,0.0,0.0,0.99940,0.9568
2,500.0,0.0,0.0,1.00000,0.8452
3,50000.0,3.0,0.0,0.98550,0.9836
4,50000.0,0.0,3.0,0.99950,0.9788
5,50000.0,3.0,3.0,0.99640,0.9803


### Выводы.

Чем меньше тестовая выборка, тем быстрее мы достигаем высоких результатов на тренировочной выборке и тем хуже результаты на проверочной (тестовой) выборке. Нужно стараться использовать как можно больше данных, но следить чтобы они были сбалансированны, очищенны и хорошо подготовленны.

drop_out показал себя просто отлично. Он конечно снизил показатель на обучаемой выборке, но это говорит только об избавлении от переобучении. А вот на тестовой выборке очень существенный прирост. Результаты прошлой недели можно было бы улучшить.

batch_normalization также дал улучшение, но не такое существенное. Их общее использование к моему удивлению также дало хороший результат. Хотя я полагал это будет слишком бессмысленно сложная нейросеть. 

Хотя у меня лучший показатель дал чистый дропаут, для улучшения показателей я бы продолжил эксперименты и с комбинацией этих методов, и с изменением гиперпараметров drop_out. Однако меня ждут мины в PRO задании))